In [1]:
import csv
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import precision_score
from xgboost.sklearn import XGBRegressor
from xgboost.sklearn import XGBClassifier
import xgboost as xgb

In [2]:
train = pd.read_csv("train.csv")
train_l = pd.read_csv("train_label.csv")
test = pd.read_csv("test.csv")

In [3]:
print(train.shape)
print(train_l.shape)
print(test.shape)

(91531, 33)
(640, 2)
(27859, 29)


In [4]:
#is_canceled', 'reservation_status_date', 'adr', 'reservation_status'
df = train.copy()
del df["is_canceled"]
del df["reservation_status_date"]
del df["adr"]
del df["reservation_status"]

In [5]:
print(df.shape)

(91531, 29)


In [6]:
for_dum = pd.concat([df, test], ignore_index=True)

In [7]:
print(for_dum.shape)

(119390, 29)


In [8]:
df2 = for_dum.copy()
del df2["arrival_date_year"]
del df2["arrival_date_week_number"]
del df2["arrival_date_day_of_month"]

In [9]:
del df2["ID"]

In [10]:
print(df2.columns)

Index(['hotel', 'lead_time', 'arrival_date_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies', 'meal',
       'country', 'market_segment', 'distribution_channel',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'deposit_type', 'agent',
       'company', 'days_in_waiting_list', 'customer_type',
       'required_car_parking_spaces', 'total_of_special_requests'],
      dtype='object')


In [11]:
def change_date(df):
    date = []
    for i in range(df.shape[0]):
        day = str(df["arrival_date_day_of_month"][i])
        if df["arrival_date_day_of_month"][i] < 10:
            day = "0" + day
        if df["arrival_date_month"][i] == "January":
            s = str(df["arrival_date_year"][i]) + "-01-" + day
        elif df["arrival_date_month"][i] == "February":
            s = str(df["arrival_date_year"][i]) + "-02-" + day
        elif df["arrival_date_month"][i] == "March":
            s = str(df["arrival_date_year"][i]) + "-03-" + day
        elif df["arrival_date_month"][i] == "April":
            s = str(df["arrival_date_year"][i]) + "-04-" + day
        elif df["arrival_date_month"][i] == "May":
            s = str(df["arrival_date_year"][i]) + "-05-" + day
        elif df["arrival_date_month"][i] == "June":
            s = str(df["arrival_date_year"][i]) + "-06-" + day
        elif df["arrival_date_month"][i] == "July":
            s = str(df["arrival_date_year"][i]) + "-07-" + day
        elif df["arrival_date_month"][i] == "August":
            s = str(df["arrival_date_year"][i]) + "-08-" + day
        elif df["arrival_date_month"][i] == "September":
            s = str(df["arrival_date_year"][i]) + "-09-" + day
        elif df["arrival_date_month"][i] == "October":
            s = str(df["arrival_date_year"][i]) + "-10-" + day
        elif df["arrival_date_month"][i] == "November":
            s = str(df["arrival_date_year"][i]) + "-11-" + day
        elif df["arrival_date_month"][i] == "December":
            s = str(df["arrival_date_year"][i]) + "-12-" + day
        date.append(s)
        
    return date

In [12]:
arr_date = change_date(train)

In [13]:
all_dum = pd.get_dummies(df2)

In [14]:
#split train and test
train_dum = all_dum[:len(train)]
test_dum = all_dum[len(train):]

### Predict is_canceled

In [26]:
l_r = [0.075, 0.1, 0.15]
n_est = [200, 500, 800, 1000]
max_d=[7, 10, 15]
sub_s = [0.85, 0.95, 1]

In [23]:
can_df = train_dum.copy()
y_can = train["is_canceled"]
x_can = can_df

In [25]:
xcan_sub, xcan_val, ycan_sub, ycan_val = train_test_split(x_can, y_can, test_size=0.1, random_state=1024)

In [30]:
best_s_can3 = 0
best_d_can3 = 0
max_score_can3 = 0

for d in max_d:
    for s in sub_s:
        # --------------------------------------------------------- #
        # train
        xgbc = XGBClassifier(booster="gbtree",
                     max_depth = d,
                     learning_rate=0.05,
                     n_estimators=200,
                     subsample = s, 
                     objective='binary:logistic', 
                    use_label_encoder =False,
                     eval_metric = 'error')
        xgbc.fit(xcan_sub, ycan_sub)

        # valid
        ypred = xgbc.predict(xcan_val)
        acc = accuracy_score(ycan_val, ypred)
        
        if acc > max_score_can3:
            print("! update best !")
            best_s_can3 = s
            best_d_can3 = d
            max_score_can3 = acc
        # --------------------------------------------------------- #
        print("s: %f, d: %d, acc: %f, \n"%(s,d,acc))
print("best s: %f, best d: %d, acc: %f"%(best_s_can3,best_d_can3,max_score_can3))

! update best !
s: 0.850000, d: 7, acc: 0.883439, 

s: 0.950000, d: 7, acc: 0.882019, 

s: 1.000000, d: 7, acc: 0.880052, 

! update best !
s: 0.850000, d: 10, acc: 0.890103, 

s: 0.950000, d: 10, acc: 0.888246, 

s: 1.000000, d: 10, acc: 0.888792, 

! update best !
s: 0.850000, d: 15, acc: 0.902993, 

s: 0.950000, d: 15, acc: 0.901573, 

s: 1.000000, d: 15, acc: 0.900262, 

best s: 0.850000, best d: 15, acc: 0.902993


In [33]:
best_l_can3 = 0
best_n_can3 = 0
max_score_can4 = 0

for l in l_r:
    for n in n_est:
        # --------------------------------------------------------- #
        # train
        xgbc = XGBClassifier(booster="gbtree",
                     max_depth = best_d_can3,
                     learning_rate=l,
                     n_estimators=n,
                     subsample = best_s_can3, 
                     objective='binary:logistic', 
                     eval_metric = 'error', 
                     use_label_encoder=False)
        xgbc.fit(xcan_sub, ycan_sub)

        # valid
        ypred = xgbc.predict(xcan_val)
        acc = accuracy_score(ycan_val, ypred)
        
        if acc > max_score_can4:
            print("! update best !")
            best_l_can3 = l
            best_n_can3 = n
            max_score_can4 = acc
        # --------------------------------------------------------- #
        print("l: %f, n: %d, score: %f, \n"%(l,n,acc))
print("best l: %f, best n: %d, score: %f"%(best_l_can3,best_n_can3,max_score_can4))

! update best !
l: 0.075000, n: 200, score: 0.903867, 

! update best !
l: 0.075000, n: 500, score: 0.906380, 

l: 0.075000, n: 800, score: 0.906380, 

! update best !
l: 0.075000, n: 1000, score: 0.907254, 

l: 0.100000, n: 200, score: 0.904741, 

l: 0.100000, n: 500, score: 0.906598, 

! update best !
l: 0.100000, n: 800, score: 0.908455, 

l: 0.100000, n: 1000, score: 0.907363, 

l: 0.150000, n: 200, score: 0.905397, 

l: 0.150000, n: 500, score: 0.905834, 

l: 0.150000, n: 800, score: 0.904960, 

l: 0.150000, n: 1000, score: 0.905178, 

best l: 0.100000, best n: 800, score: 0.908455


### Predict adr

In [17]:
l_r = [0.01, 0.05, 0.075]
n_est = [500, 800, 1000]
max_d=[10, 15, 20]
sub_s = [0.85, 0.95, 1]

In [15]:
adr_df = train_dum.copy()
y_adr = train["adr"]
x_adr = adr_df

In [16]:
xadr_sub, xadr_val, yadr_sub, yadr_val = train_test_split(x_adr, y_adr, test_size=0.1, random_state=1024)

In [18]:
best_s = 0
best_d = 0
max_score = 0

for d in max_d:
    for s in sub_s:
        # --------------------------------------------------------- #
        # train
        xgbc = XGBRegressor(booster="gbtree",
                     max_depth = d,
                     learning_rate=0.05,
                     n_estimators=200,
                     subsample = s)
        xgbc.fit(xadr_sub, yadr_sub)

        # valid
        ypred = xgbc.predict(xadr_val)
        score = xgbc.score(xadr_val, yadr_val)  
        
        if score > max_score:
            print("! update best !")
            best_s = s
            best_d = d
            max_score = score
        # --------------------------------------------------------- #
        print("s: %f, d: %d, score: %f, \n"%(s,d,score))
print("best s: %f, best d: %d, score: %f"%(best_s,best_d,max_score))

! update best !
s: 0.850000, d: 10, score: 0.838788, 

! update best !
s: 0.950000, d: 10, score: 0.839978, 

s: 1.000000, d: 10, score: 0.838127, 

! update best !
s: 0.850000, d: 15, score: 0.844518, 

s: 0.950000, d: 15, score: 0.842938, 

s: 1.000000, d: 15, score: 0.841184, 

s: 0.850000, d: 20, score: 0.840434, 

s: 0.950000, d: 20, score: 0.837881, 

s: 1.000000, d: 20, score: 0.831864, 

best s: 0.850000, best d: 15, score: 0.844518


In [19]:
best_l = 0
best_n = 0
max_score2 = 0

for l in l_r:
    for n in n_est:
        # --------------------------------------------------------- #
        # train
        xgbc = XGBRegressor(booster="gbtree",
                     max_depth = best_d,
                     learning_rate = l,
                     n_estimators = n,
                     subsample = best_s)
        xgbc.fit(xadr_sub, yadr_sub)

        # valid
        ypred = xgbc.predict(xadr_val)
        score = xgbc.score(xadr_val, yadr_val)  
        
        if score > max_score2:
            print("! update best !")
            best_l = l
            best_n = n
            max_score2 = score
        # --------------------------------------------------------- #
        print("l: %f, n: %d, score: %f, \n"%(l,n,score))
print("best l: %f, best n: %d, score: %f"%(best_l,best_n,max_score2))

! update best !
l: 0.010000, n: 500, score: 0.842510, 

! update best !
l: 0.010000, n: 800, score: 0.846258, 

! update best !
l: 0.010000, n: 1000, score: 0.846906, 

l: 0.050000, n: 500, score: 0.845875, 

l: 0.050000, n: 800, score: 0.846184, 

l: 0.050000, n: 1000, score: 0.846302, 

l: 0.075000, n: 500, score: 0.843831, 

l: 0.075000, n: 800, score: 0.843830, 

l: 0.075000, n: 1000, score: 0.843770, 

best l: 0.010000, best n: 1000, score: 0.846906


### Test

In [20]:
best_d_can3 = 15
best_l_can3 = 0.1
best_n_can3 = 800
best_s_can3 = 0.85

In [21]:
arr_date_test = change_date(test)

In [25]:
xgbc = XGBClassifier(booster="gbtree",
                     max_depth = best_d_can3,
                     learning_rate=best_l_can3,
                     n_estimators=best_n_can3,
                     subsample = best_s_can3, 
                     objective='binary:logistic', 
                     eval_metric = 'error',
                     use_label_encoder=False)
xgbc.fit(x_can, y_can)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='error',
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.1, max_delta_step=0,
              max_depth=15, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=800, n_jobs=8,
              num_parallel_tree=1, random_state=0, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, subsample=0.85, tree_method='exact',
              use_label_encoder=False, validate_parameters=1, verbosity=None)

In [43]:
feature_name = []
for col,score in zip(x_can.columns, xgbc.feature_importances_):
    feature_name.append((col,score))
sorted_coef = sorted(feature_name, key=lambda x: abs(x[1]), reverse = True)

In [44]:
print(sorted_coef[:20])

[('deposit_type_Non Refund', 0.7385), ('required_car_parking_spaces', 0.055864763), ('previous_cancellations', 0.036617458), ('reserved_room_type_P', 0.007558049), ('market_segment_Online TA', 0.0074155414), ('country_PRT', 0.005071911), ('previous_bookings_not_canceled', 0.004255408), ('country_AGO', 0.0042289817), ('assigned_room_type_I', 0.004085085), ('customer_type_Transient', 0.00266635), ('country_ARE', 0.002239956), ('country_NGA', 0.002205459), ('country_MAC', 0.0020808452), ('market_segment_Groups', 0.0019851793), ('country_HKG', 0.0019776828), ('meal_Undefined', 0.0019171964), ('market_segment_Direct', 0.0018619441), ('country_GIB', 0.0016986111), ('country_DEU', 0.0016860868), ('country_PAN', 0.0016853147)]


In [26]:
ypred_can = xgbc.predict(test_dum)

In [27]:
print(sum(ypred_can))

8920


In [28]:
xgbr = XGBRegressor(booster="gbtree",
                     max_depth = best_d,
                     learning_rate = best_l,
                     n_estimators = best_n,
                     subsample = best_s)
xgbr.fit(x_adr, y_adr)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.01, max_delta_step=0, max_depth=15,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.85,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [45]:
feature_name = []
for col,score in zip(x_adr.columns, xgbr.feature_importances_):
    feature_name.append((col,score))
sorted_coef = sorted(feature_name, key=lambda x: abs(x[1]), reverse = True)

In [46]:
print(sorted_coef[:20])

[('deposit_type_No Deposit', 0.12405396), ('hotel_City Hotel', 0.10969548), ('market_segment_Online TA', 0.102983356), ('arrival_date_month_August', 0.07405902), ('arrival_date_month_July', 0.051747773), ('market_segment_Direct', 0.039138056), ('reserved_room_type_A', 0.03576105), ('reserved_room_type_F', 0.031035226), ('market_segment_Complementary', 0.030199105), ('deposit_type_Non Refund', 0.02477271), ('arrival_date_month_September', 0.017992685), ('children', 0.017635792), ('meal_HB', 0.013165229), ('reserved_room_type_B', 0.01242564), ('assigned_room_type_G', 0.012389966), ('assigned_room_type_L', 0.011109045), ('reserved_room_type_G', 0.010584361), ('arrival_date_month_June', 0.010349853), ('arrival_date_month_March', 0.009913133), ('market_segment_Groups', 0.008647074)]


In [29]:
ypred_adr = xgbr.predict(test_dum)

In [30]:
op_test = [1]*test.shape[0]
s_test = ypred_adr*(op_test - ypred_can)*(test["stays_in_weekend_nights"] + test["stays_in_week_nights"])
arr_df_test = pd.DataFrame(arr_date_test)
cat_test = pd.concat([arr_df_test, s_test], axis = 1)
cat_test.columns = ["arrDate", "rev"]
print(cat_test)

          arrDate          rev
0      2017-04-01   724.908447
1      2017-04-01     0.000000
2      2017-04-01   414.098316
3      2017-04-01   890.358215
4      2017-04-01     0.000000
...           ...          ...
27854  2017-08-31  1183.146866
27855  2017-08-31  1720.433609
27856  2017-08-31   734.033661
27857  2017-08-31   650.269142
27858  2017-08-31    73.686676

[27859 rows x 2 columns]


In [31]:
test_nl = pd.read_csv("test_nolabel.csv")

In [32]:
def unique(list1): 
    unique_list = [] 
    for x in list1: 
        # check if exists in unique_list or not 
        if x not in unique_list: 
            unique_list.append(x) 
    return unique_list

In [33]:
uni_date_test = unique(arr_date_test)
print(len(uni_date_test))
print(len(test_nl))

153
153


In [34]:
blank_test = [0]*len(uni_date_test)
#sum up
for i in range(cat_test.shape[0]):
    for j in range(len(uni_date_test)):
        if cat_test["arrDate"][i] == uni_date_test[j]:
            blank_test[j] += cat_test["rev"][i]
            break

In [35]:
import math

In [36]:
pre_class = blank_test.copy()
for i in range(len(uni_date_test)):
    pre_class[i] = math.floor(pre_class[i] / 10000)

In [37]:
print(pre_class)

[3, 2, 4, 1, 3, 2, 2, 4, 4, 3, 3, 2, 4, 3, 3, 4, 3, 1, 3, 2, 3, 3, 3, 3, 1, 1, 2, 5, 4, 2, 4, 3, 2, 3, 4, 4, 3, 4, 3, 2, 4, 2, 3, 4, 3, 2, 2, 3, 3, 3, 4, 3, 2, 3, 4, 3, 2, 4, 3, 2, 2, 4, 5, 4, 4, 5, 2, 2, 4, 3, 4, 6, 4, 2, 2, 4, 3, 4, 4, 4, 3, 3, 3, 3, 3, 3, 4, 3, 2, 3, 3, 6, 4, 6, 5, 3, 4, 4, 5, 6, 6, 5, 3, 4, 5, 8, 6, 7, 4, 3, 4, 4, 6, 7, 7, 4, 4, 4, 4, 7, 5, 6, 5, 4, 6, 5, 6, 6, 8, 4, 4, 4, 4, 7, 6, 6, 4, 5, 7, 6, 6, 6, 6, 3, 4, 4, 4, 6, 5, 7, 3, 3, 4]


In [38]:
ypred_df = pd.DataFrame(pre_class)
date_df = pd.DataFrame(uni_date_test)
final = pd.concat([date_df, ypred_df], axis = 1)
final.columns = ["arrival_date", "label"]
display(final)

,arrival_date,label
0,2017-04-01,3
1,2017-04-02,2
2,2017-04-03,4
3,2017-04-04,1
4,2017-04-05,3
...,...,...
148,2017-08-27,5
149,2017-08-28,7
150,2017-08-29,3
151,2017-08-30,3


In [39]:
final.to_csv("test_label.csv", index = False, header = True)